In [1]:
import os
import math
import numpy as np
import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector
from datetime import datetime, date, time

batch_size = 64
embedding_dimension = 5
negative_samples = 8

# 텐서보드 모델 요약을 저장할 위치
LOG_DIR = "/home/younseun/columbus_study/04_learning_tensorflow/06__word_embeddings_and_rnns/logs/word2vec_intro_"+datetime.now().strftime('%Y%m%d%H%M%S')
LOG_DIR = "/tmp/logs/word2vec_intro"

digit_to_word_map = {1: "One", 2: "Two", 3: "Three", 4: "Four", 5: "Five",
                     6: "Six", 7: "Seven", 8: "Eight", 9: "Nine"}
sentences = []

# Create two kinds of sentences - sequences of odd and even digits.
for i in range(10000):
    rand_odd_ints = np.random.choice(range(1, 10, 2), 3)
    sentences.append(" ".join([digit_to_word_map[r] for r in rand_odd_ints]))
    rand_even_ints = np.random.choice(range(2, 10, 2), 3)
    sentences.append(" ".join([digit_to_word_map[r] for r in rand_even_ints]))

# 하나만 테스트로 진행할때 사용
# sentences = []
# sentences.append("Eight Four Six")

In [2]:
sentences[:10]

['Nine One Three',
 'Eight Six Eight',
 'One One Nine',
 'Four Six Six',
 'One Five Three',
 'Eight Six Six',
 'Nine Nine Nine',
 'Four Two Two',
 'Nine Seven Five',
 'Four Two Six']

In [3]:
# 단어를 인덱스에 매핑
word2index_map = {}
index = 0
for sent in sentences:
    #print(sent)
    for word in sent.lower().split():
        #print(word)
        if word not in word2index_map:
            word2index_map[word] = index
            index += 1
        #print(word2index_map)

# 역방향 매핑
index2word_map = {index: word for word, index in word2index_map.items()}
vocabulary_size = len(index2word_map)

print('word2index_map :{}'.format(word2index_map))
print('index2word_map :{}'.format(index2word_map))
print('vocabulary_size :{}'.format(vocabulary_size))


word2index_map :{'nine': 0, 'one': 1, 'three': 2, 'eight': 3, 'six': 4, 'four': 5, 'five': 6, 'two': 7, 'seven': 8}
index2word_map :{0: 'nine', 1: 'one', 2: 'three', 3: 'eight', 4: 'six', 5: 'four', 6: 'five', 7: 'two', 8: 'seven'}
vocabulary_size :9


In [4]:
# Generate skip-gram pairs
sample_seq = 0
skip_gram_pairs = []
for sent in sentences:
    tokenized_sent = sent.lower().split()
    sample_seq += 1
    if sample_seq == 1:
        print(tokenized_sent)
    for i in range(1, len(tokenized_sent)-1):
        word_context_pair = [[word2index_map[tokenized_sent[i-1]],
                              word2index_map[tokenized_sent[i+1]]],
                             word2index_map[tokenized_sent[i]]]
        skip_gram_pairs.append([word_context_pair[1],
                                word_context_pair[0][0]])
        skip_gram_pairs.append([word_context_pair[1],
                                word_context_pair[0][1]])
        if sample_seq == 10:
            print(word_context_pair)
            print([word_context_pair[1], word_context_pair[0][0]])
            print([word_context_pair[1], word_context_pair[0][1]])
    

['nine', 'one', 'three']
[[5, 4], 7]
[7, 5]
[7, 4]


In [5]:
def get_skipgram_batch(batch_size):
    instance_indices = list(range(len(skip_gram_pairs)))
    np.random.shuffle(instance_indices)
    batch = instance_indices[:batch_size]
    x = [skip_gram_pairs[i][0] for i in batch]      # Target word
    y = [[skip_gram_pairs[i][1]] for i in batch]    # 문맥 단어
    return x, y

In [6]:
#skip_gram_pairs 데이터 확인
x, y = get_skipgram_batch(batch_size)
print('x:{}, y:{}'.format(x[:3],y[:3]))
print('x(target word) : ', [index2word_map[index] for index in x[:3]])
print('y(context word) : ', [index2word_map[index[0]] for index in y[:3]])


x:[8, 3, 2], y:[[1], [4], [0]]
x(target word) :  ['seven', 'eight', 'three']
y(context word) :  ['one', 'six', 'nine']


In [7]:
# Input data, labels
train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])

In [8]:
# Embedding lookup table currently only implemented in CPU
with tf.name_scope("embeddings"):
    embeddings = tf.Variable(
        tf.random_uniform([vocabulary_size, embedding_dimension],
                          -1.0, 1.0), name='embedding')
    # This is essentialy a lookup table
    embed = tf.nn.embedding_lookup(embeddings, train_inputs)
embeddings, train_inputs, embed

(<tf.Variable 'embeddings/embedding:0' shape=(9, 5) dtype=float32_ref>,
 <tf.Tensor 'Placeholder:0' shape=(64,) dtype=int32>,
 <tf.Tensor 'embeddings/embedding_lookup:0' shape=(64, 5) dtype=float32>)

In [9]:
# Create variables for the NCE loss
nce_weights = tf.Variable(
        tf.truncated_normal([vocabulary_size, embedding_dimension],
                            stddev=1.0 / math.sqrt(embedding_dimension)))
nce_biases = tf.Variable(tf.zeros([vocabulary_size]))


loss = tf.reduce_mean(tf.nn.nce_loss(weights=nce_weights, biases=nce_biases, inputs=embed, labels=train_labels,
                      num_sampled=negative_samples, num_classes=vocabulary_size))
tf.summary.scalar("NCE_loss", loss)


<tf.Tensor 'NCE_loss_1:0' shape=() dtype=string>

In [10]:
# Learning rate decay
global_step = tf.Variable(0, trainable=False)
learningRate = tf.train.exponential_decay(learning_rate=0.1,
                                          global_step=global_step,
                                          decay_steps=1000,
                                          decay_rate=0.95,
                                          staircase=True)
train_step = tf.train.GradientDescentOptimizer(learningRate).minimize(loss)

merged = tf.summary.merge_all()

In [11]:
first_onoff = 0
with tf.Session() as sess:
    train_writer = tf.summary.FileWriter(LOG_DIR,
                                         graph=tf.get_default_graph())
    saver = tf.train.Saver()

    with open(os.path.join(LOG_DIR, 'metadata.tsv'), "w") as metadata:
        metadata.write('Name\tClass\n')
        for k, v in index2word_map.items():
            metadata.write('%s\t%d\n' % (v, k))

    config = projector.ProjectorConfig()
    embedding = config.embeddings.add()
    embedding.tensor_name = embeddings.name
    # Link this tensor to its metadata file (e.g. labels).
    embedding.metadata_path = os.path.join(LOG_DIR, 'metadata.tsv')
    projector.visualize_embeddings(train_writer, config)

    tf.global_variables_initializer().run()

    for step in range(1000):
        x_batch, y_batch = get_skipgram_batch(batch_size)
        first_onoff += 1
        if first_onoff == 1:
            print(x_batch, y_batch)
        summary, _ = sess.run([merged, train_step],
                              feed_dict={train_inputs: x_batch,
                                         train_labels: y_batch})
        train_writer.add_summary(summary, step)

        if step % 100 == 0:
            saver.save(sess, os.path.join(LOG_DIR, "w2v_model.ckpt"), step)
            loss_value = sess.run(loss,
                                  feed_dict={train_inputs: x_batch,
                                             train_labels: y_batch})
            print("Loss at %d: %.5f" % (step, loss_value))

    # Normalize embeddings before using
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
    normalized_embeddings = embeddings / norm
    normalized_embeddings_matrix = sess.run(normalized_embeddings)
print("completed !!")

[2, 1, 4, 5, 6, 3, 3, 3, 1, 6, 5, 5, 1, 2, 6, 2, 2, 7, 4, 4, 8, 4, 0, 5, 6, 4, 6, 7, 1, 8, 2, 8, 3, 7, 1, 8, 5, 8, 1, 6, 4, 0, 7, 0, 4, 1, 4, 7, 2, 3, 6, 5, 0, 3, 8, 0, 5, 8, 5, 1, 1, 3, 1, 1] [[0], [6], [5], [3], [0], [5], [4], [5], [2], [2], [7], [5], [8], [6], [2], [0], [6], [3], [7], [7], [2], [4], [8], [5], [6], [4], [8], [4], [0], [0], [6], [0], [5], [7], [6], [6], [4], [6], [2], [6], [3], [2], [5], [0], [7], [8], [3], [3], [6], [3], [2], [4], [8], [3], [1], [6], [3], [2], [3], [1], [1], [5], [2], [2]]
Loss at 0: 6.64638
Loss at 100: 3.12801
Loss at 200: 2.77991
Loss at 300: 2.62005
Loss at 400: 2.59635
Loss at 500: 2.58121
Loss at 600: 2.51445
Loss at 700: 2.54663
Loss at 800: 2.53083
Loss at 900: 2.52549
Instructions for updating:
keep_dims is deprecated, use keepdims instead
completed !!


In [12]:
ref_word = normalized_embeddings_matrix[word2index_map["one"]]

cosine_dists = np.dot(normalized_embeddings_matrix, ref_word)
ff = np.argsort(cosine_dists)[::-1][1:10]
for f in ff:
    print(index2word_map[f])
    print(cosine_dists[f])


seven
0.96873075
five
0.9580324
three
0.9303351
nine
0.8212328
eight
-0.055619687
six
-0.0755487
two
-0.09833431
four
-0.105676994


In [13]:
word2index_map["one"]

1

In [14]:
ref_word

array([ 0.14506422, -0.4546689 , -0.23660548,  0.17448369,  0.8281338 ],
      dtype=float32)

In [15]:
normalized_embeddings_matrix

array([[ 0.25361255, -0.22201024,  0.21316423, -0.11294982,  0.9100525 ],
       [ 0.14506422, -0.4546689 , -0.23660548,  0.17448369,  0.8281338 ],
       [-0.02065117, -0.46379608, -0.2746809 ,  0.47800875,  0.6931989 ],
       [-0.8430958 ,  0.1733758 , -0.318345  , -0.36302474,  0.16124405],
       [-0.9168602 ,  0.22670692, -0.30922624,  0.06155199,  0.0925298 ],
       [-0.6046285 ,  0.28968465, -0.05042597, -0.68991154,  0.26830304],
       [ 0.28740245, -0.25902125, -0.27318826,  0.02839938,  0.8802667 ],
       [-0.84565794,  0.27027208, -0.1637651 , -0.3753237 ,  0.21006854],
       [ 0.24593292, -0.31342536, -0.12484634,  0.05732991,  0.90686727]],
      dtype=float32)

In [18]:
for k, v in index2word_map.items():
            print('{} {}'.format(v, k))

nine 0
one 1
three 2
eight 3
six 4
four 5
five 6
two 7
seven 8


In [19]:
embeddings.name

'embeddings/embedding:0'